# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 13 2022 12:14PM,248560,19,ADV80006508,"AdvaGen Pharma, Ltd",Released
1,Oct 13 2022 12:14PM,248560,19,ADV80006509,"AdvaGen Pharma, Ltd",Executing
2,Oct 13 2022 12:14PM,248560,19,ADV80006510,"AdvaGen Pharma, Ltd",Released
3,Oct 13 2022 12:14PM,248560,19,ADV80006511,"AdvaGen Pharma, Ltd",Released
4,Oct 13 2022 12:14PM,248560,19,ADV80006515,"AdvaGen Pharma, Ltd",Released
5,Oct 13 2022 12:14PM,248560,19,ADV80006516,"AdvaGen Pharma, Ltd",Released
6,Oct 13 2022 11:42AM,248555,12,HH8162,Hush Hush,Released
7,Oct 13 2022 11:22AM,248553,16,8638286,Sartorius Bioprocess Solutions,Released
8,Oct 13 2022 10:48AM,248549,19,TR-RIH-10142022,"GCH Granules USA, Inc.",Released
9,Oct 13 2022 10:35AM,248266,15,PNC391300B,"Person & Covey, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,248549,Released,1
20,248553,Released,1
21,248555,Released,1
22,248560,Executing,1
23,248560,Released,5


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248541,NaN,3.0,NaN
248549,NaN,NaN,1.0
248553,NaN,NaN,1.0
248555,NaN,NaN,1.0
248560,NaN,1.0,5.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248089,0.0,0.0,1.0
248266,0.0,0.0,30.0
248495,0.0,2.0,0.0
248510,6.0,7.0,1.0
248529,5.0,5.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248089,0,0,1
248266,0,0,30
248495,0,2,0
248510,6,7,1
248529,5,5,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248089,0,0,1
1,248266,0,0,30
2,248495,0,2,0
3,248510,6,7,1
4,248529,5,5,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248089,,,1
1,248266,,,30
2,248495,,2,
3,248510,6,7,1
4,248529,5,5,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 13 2022 12:14PM,248560,19,"AdvaGen Pharma, Ltd"
6,Oct 13 2022 11:42AM,248555,12,Hush Hush
7,Oct 13 2022 11:22AM,248553,16,Sartorius Bioprocess Solutions
8,Oct 13 2022 10:48AM,248549,19,"GCH Granules USA, Inc."
9,Oct 13 2022 10:35AM,248266,15,"Person & Covey, Inc."
39,Oct 13 2022 10:31AM,248540,15,"Alliance Pharma, Inc."
84,Oct 13 2022 10:28AM,248541,10,Emerginnova
87,Oct 13 2022 10:20AM,248538,10,Bio-PRF
104,Oct 13 2022 10:17AM,248534,10,Eywa Pharma Inc.
123,Oct 13 2022 9:47AM,248532,19,"NAPP Technologies, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 13 2022 12:14PM,248560,19,"AdvaGen Pharma, Ltd",,1,5
1,Oct 13 2022 11:42AM,248555,12,Hush Hush,,,1
2,Oct 13 2022 11:22AM,248553,16,Sartorius Bioprocess Solutions,,,1
3,Oct 13 2022 10:48AM,248549,19,"GCH Granules USA, Inc.",,,1
4,Oct 13 2022 10:35AM,248266,15,"Person & Covey, Inc.",,,30
5,Oct 13 2022 10:31AM,248540,15,"Alliance Pharma, Inc.",,27,18
6,Oct 13 2022 10:28AM,248541,10,Emerginnova,,3,
7,Oct 13 2022 10:20AM,248538,10,Bio-PRF,,5,12
8,Oct 13 2022 10:17AM,248534,10,Eywa Pharma Inc.,,9,10
9,Oct 13 2022 9:47AM,248532,19,"NAPP Technologies, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 13 2022 12:14PM,248560,19,"AdvaGen Pharma, Ltd",5,1,
1,Oct 13 2022 11:42AM,248555,12,Hush Hush,1,,
2,Oct 13 2022 11:22AM,248553,16,Sartorius Bioprocess Solutions,1,,
3,Oct 13 2022 10:48AM,248549,19,"GCH Granules USA, Inc.",1,,
4,Oct 13 2022 10:35AM,248266,15,"Person & Covey, Inc.",30,,
5,Oct 13 2022 10:31AM,248540,15,"Alliance Pharma, Inc.",18,27,
6,Oct 13 2022 10:28AM,248541,10,Emerginnova,,3,
7,Oct 13 2022 10:20AM,248538,10,Bio-PRF,12,5,
8,Oct 13 2022 10:17AM,248534,10,Eywa Pharma Inc.,10,9,
9,Oct 13 2022 9:47AM,248532,19,"NAPP Technologies, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 13 2022 12:14PM,248560,19,"AdvaGen Pharma, Ltd",5,1,
1,Oct 13 2022 11:42AM,248555,12,Hush Hush,1,,
2,Oct 13 2022 11:22AM,248553,16,Sartorius Bioprocess Solutions,1,,
3,Oct 13 2022 10:48AM,248549,19,"GCH Granules USA, Inc.",1,,
4,Oct 13 2022 10:35AM,248266,15,"Person & Covey, Inc.",30,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 13 2022 12:14PM,248560,19,"AdvaGen Pharma, Ltd",5.0,1.0,NaN
1,Oct 13 2022 11:42AM,248555,12,Hush Hush,1.0,NaN,NaN
2,Oct 13 2022 11:22AM,248553,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
3,Oct 13 2022 10:48AM,248549,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
4,Oct 13 2022 10:35AM,248266,15,"Person & Covey, Inc.",30.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 13 2022 12:14PM,248560,19,"AdvaGen Pharma, Ltd",5.0,1.0,0.0
1,Oct 13 2022 11:42AM,248555,12,Hush Hush,1.0,0.0,0.0
2,Oct 13 2022 11:22AM,248553,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
3,Oct 13 2022 10:48AM,248549,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
4,Oct 13 2022 10:35AM,248266,15,"Person & Covey, Inc.",30.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1739678,53.0,32.0,11.0
12,248555,1.0,0.0,0.0
15,496806,48.0,27.0,0.0
16,248553,1.0,0.0,0.0
19,745641,7.0,1.0,0.0
21,248089,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1739678,53.0,32.0,11.0
1,12,248555,1.0,0.0,0.0
2,15,496806,48.0,27.0,0.0
3,16,248553,1.0,0.0,0.0
4,19,745641,7.0,1.0,0.0
5,21,248089,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,53.0,32.0,11.0
1,12,1.0,0.0,0.0
2,15,48.0,27.0,0.0
3,16,1.0,0.0,0.0
4,19,7.0,1.0,0.0
5,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,53.0
1,12,Released,1.0
2,15,Released,48.0
3,16,Released,1.0
4,19,Released,7.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21
Status,,,,,,
Completed,11.0,0.0,0.0,0.0,0.0,0.0
Executing,32.0,0.0,27.0,0.0,1.0,0.0
Released,53.0,1.0,48.0,1.0,7.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21
0,Completed,11.0,0.0,0.0,0.0,0.0,0.0
1,Executing,32.0,0.0,27.0,0.0,1.0,0.0
2,Released,53.0,1.0,48.0,1.0,7.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21
0,Completed,11.0,0.0,0.0,0.0,0.0,0.0
1,Executing,32.0,0.0,27.0,0.0,1.0,0.0
2,Released,53.0,1.0,48.0,1.0,7.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()